In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from tqdm import tqdm

In [2]:
input_file_path = '../../../siads591 data/processed_data/gabbard/gabbard_1D_scaled_1980.pkl'
output_file_path = './gabbard_debris_1D_60fps_1980_start.mp4'

In [3]:
bitrate = 1000
dpi = 300
fps = 60
freq = '1D'
use_slice = slice(None, None, None) # 2 ways to slice the data, this is index slicing for the date range

In [4]:
df = pd.read_pickle(f'{input_file_path}')
df = df[(df.index >= "1980-01-01") & (df.index <= "2021-01-20")] # manually set min/max range based on datetime
display(df.info())
display(df.head(4))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 79734374 entries, 1980-01-01 to 2021-01-20
Data columns (total 4 columns):
 #   Column        Dtype  
---  ------        -----  
 0   NORAD_CAT_ID  uint64 
 1   PERIOD        float64
 2   APOAPSIS      float64
 3   PERIAPSIS     float64
dtypes: float64(3), uint64(1)
memory usage: 3.0 GB


None

,NORAD_CAT_ID,PERIOD,APOAPSIS,PERIAPSIS
EPOCH,,,,
1980-01-01,51,59122.0,33706.200000,30377.800000
1980-01-02,51,59122.0,33705.416667,30378.527778
1980-01-03,51,59122.0,33704.833333,30379.055556
1980-01-04,51,59122.0,33704.250000,30379.583333


In [5]:
dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)
dates = dates[use_slice]
print(f'{len(dates)} frames')

14996 frames


In [ ]:
# set this to a specific frame number to draw only 1 frame in the notebook to test output
# Set None generate full video
draw_one_frame = None

fig, ax = plt.subplots(dpi=dpi, figsize=(8,5))

ax.set_xlim(87,130)
ax.set_ylim(100,2750)
ax.set_xlabel("Orbital Period (min)")
ax.set_ylabel("Altitude (km)")
artists = []

def plot_it(ax,e):
    global df
    cdf = df[df.index==e].copy()

    # revert the scaling of the data
    cdf['PERIOD'] = cdf.PERIOD.astype(np.float64) / 500
    cdf['APOAPSIS'] = cdf.APOAPSIS.astype(np.float64) / 20
    cdf['PERIAPSIS'] = cdf.PERIAPSIS.astype(np.float64) / 20
    
    apo = ax.scatter(cdf.PERIOD, cdf.APOAPSIS, color='#fe2700', marker='o', edgecolors='none', alpha=0.25, s=1, label="Apoapsis")
    peri = ax.scatter(cdf.PERIOD, cdf.PERIAPSIS, color='#038fd5', marker='o', edgecolors='none', alpha=0.25, s=1, label="Periapsis")
    plt.legend(loc="upper right", numpoints=1, fontsize=10, markerscale=10)
    return [apo,peri]

def update(f):
    global pbar, artists, yr
    i, date = f
    ax.set_title(f"{date.month_name()} {date.year} (frame {i})", loc='right')
    for a in artists:
        a.remove()
    artists = plot_it(ax,date)
    pbar.update(1)

if type(draw_one_frame) == int:
    pbar = tqdm(total=1, position=0, leave=True)
    update((draw_one_frame,dates[draw_one_frame]))
    pbar.close()
else:
    pbar = tqdm(total=len(dates)+1, position=0, leave=True)
    ani = FuncAnimation(fig, update, list(enumerate(dates)))
    writervideo = FFMpegWriter(fps=fps, bitrate=bitrate)
    ani.save(f'{output_file_path}', writer=writervideo, dpi=dpi)
    pbar.close()

 32%|███▏      | 4749/14997 [22:23<48:22,  3.53it/s]  